In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
apply_train_df = pd.read_csv('apply_train.csv')

In [ ]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
user_item_matrix[user_item_matrix > 1] = 1

# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_item_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

In [ ]:
user_similarity.shape

(8482, 8482)

In [36]:
user_predicted_scores.shape

(8482, 6695)

In [35]:
recommendations = []
sorted_indices = np.argsort(user_similarity, axis=1)

for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)

    # 해당 사용자의 유사도가 높은 다른 사용자의 인덱스
    similar_users = sorted_indices[idx, 1:]  # 자기 자신 제외
    print(similar_users)

    for similar_user_idx in similar_users:
        similar_user = user_item_matrix.index[similar_user_idx]
        recommended_jobs = [job for job in user_item_matrix.columns if job not in applied_jobs][:5]

        for job in recommended_jobs:
            recommendations.append([user, job])

[5659 5658 5657 ... 7058  653  615]


KeyboardInterrupt: ignored

In [ ]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)